<table>
 <td>All code is written by Joo Won Park, Columbia University (jp3378@columbia.edu)</td>
</table>

In [1]:
#Loading libraries
import numpy as np
import IPython
from scipy.io.wavfile import read
from scipy.io.wavfile import write
from scipy import signal
import scipy.io as sio

* Load the desired audio file (mono) in place of 'drumloop.wav'. The file should be located in
the same folder as this python notebook file is. 

* For this code, you need HRTF.mat file that is provided by professor Mesgarani.

* Running this code will produce 24 audio files. Each file will be convolved so that that the whole set of file pans the horizontal plane by 15 degrees. The names of file will be 'drumHRIR_0', ...,'drumHRIR_23',


In [2]:
drumloop = read("drumloop.wav")
drum_data, drum_f = drumloop[1], drumloop[0]
time = np.linspace(0, len(drum_data)/drum_f, num=len(drum_data))

HRIR = sio.loadmat('HRTF.mat')
evlist = HRIR['evlist']
azlist = HRIR['azlist']

rcontent = HRIR['rcontentm']
lcontent = HRIR['lcontentm']



In [3]:
ev0 = np.where(evlist==0)[0] #index where 0 elevation 

azonly = azlist[ev0]
ev0_l = lcontent[ev0,:]
ev0_r = rcontent[ev0,:]

# create dictionary of convolved data
d={}

for i in xrange(0,len(azonly)):
        d["new_{0}".format(i)]=np.vstack((signal.convolve(drum_data,lcontent[i],mode='same'),
                                          signal.convolve(drum_data,rcontent[i],mode='same')))



def scaleandwrite(outputfile, samplerate, drumdata):
    scaled = np.int16((drumdata)/np.max(np.abs(drumdata)) * 32767)
    write(outputfile,rate = samplerate, data = scaled.T)

    
for i in xrange(0,len(azonly)):
    scaleandwrite('drumHRIR_{0}.wav'.format(i), drum_f, d['new_{0}'.format(i)])
    